In [42]:
from pyelasticsearch.client import ElasticSearch
import json
import requests
import pandas as pd
import math
import numpy as np
import boto3
import re

In [3]:
config = json.load(open('../config/configsql.json', 'r'))
aws_access_key_id=config['dev']['aws']['key']
aws_secret_access_key=config['dev']['aws']['secret']
region = 'us-east-1'
endpoint = config['dev']['endpoint']
bucketname=config['dev']['bucketname']['bucketname']
s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
index_name1 = 'candidates_cv'
index_name2 = 'parsed_resume'
es = ElasticSearch(endpoint, port=443)

In [16]:
#id = 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_aee1411d.pdf.txt'

In [137]:
d_all = es.search(index = index_name2, query={
                                     "query": {"match_all": {}},
                                     "size": 2
                                      })

In [143]:
all_user = {}
for i in range(len(d_all['hits']['hits'])):
    user = d_all['hits']['hits'][i]['_source']
    id = d_all['hits']['hits'][i]['_id']
    all_user[id] = user

In [144]:
user

{'education': {'entities': {'Entities': [{'BeginOffset': 0,
     'EndOffset': 21,
     'Score': 0.6793285012245178,
     'Text': 'Bachlors of Computing',
     'Type': 'TITLE'},
    {'BeginOffset': 23,
     'EndOffset': 30,
     'Score': 0.591768205165863,
     'Text': 'Honours',
     'Type': 'TITLE'},
    {'BeginOffset': 42,
     'EndOffset': 46,
     'Score': 0.9950048327445984,
     'Text': '2016',
     'Type': 'DATE'},
    {'BeginOffset': 51,
     'EndOffset': 69,
     'Score': 0.8884028196334839,
     'Text': "Queen's University",
     'Type': 'ORGANIZATION'},
    {'BeginOffset': 73,
     'EndOffset': 79,
     'Score': 0.9988804459571838,
     'Text': 'Canada',
     'Type': 'LOCATION'}],
   'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
     'content-length': '507',
     'content-type': 'application/x-amz-json-1.1',
     'date': 'Thu, 15 Mar 2018 23:11:49 GMT',
     'x-amzn-requestid': '3da0a36b-28a6-11e8-846c-8318cd5d2a9f'},
    'HTTPStatusCode': 200,
    'Reques

In [145]:
for i in range(len(d_all['hits']['hits'])):
    user = d_all['hits']['hits'][i]

In [146]:
user['_source']

{'education': {'entities': {'Entities': [{'BeginOffset': 0,
     'EndOffset': 21,
     'Score': 0.6793285012245178,
     'Text': 'Bachlors of Computing',
     'Type': 'TITLE'},
    {'BeginOffset': 23,
     'EndOffset': 30,
     'Score': 0.591768205165863,
     'Text': 'Honours',
     'Type': 'TITLE'},
    {'BeginOffset': 42,
     'EndOffset': 46,
     'Score': 0.9950048327445984,
     'Text': '2016',
     'Type': 'DATE'},
    {'BeginOffset': 51,
     'EndOffset': 69,
     'Score': 0.8884028196334839,
     'Text': "Queen's University",
     'Type': 'ORGANIZATION'},
    {'BeginOffset': 73,
     'EndOffset': 79,
     'Score': 0.9988804459571838,
     'Text': 'Canada',
     'Type': 'LOCATION'}],
   'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
     'content-length': '507',
     'content-type': 'application/x-amz-json-1.1',
     'date': 'Thu, 15 Mar 2018 23:11:49 GMT',
     'x-amzn-requestid': '3da0a36b-28a6-11e8-846c-8318cd5d2a9f'},
    'HTTPStatusCode': 200,
    'Reques

In [157]:
def get_user_education(user):
    alls = {}
    education = user['_source']['education']
    id = user['_id']
    text = [i['Text'] for i in user['_source']['education']['entities']['Entities']]
    type = [i['Type'] for i in user['_source']['education']['entities']['Entities']]
    my_dict = defaultdict(list)
    for k, v in zip(type, text):
        my_dict[k].append(v)
    alls = {'id': id, "dict":my_dict}
    return alls

In [158]:
user

{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_aee1411d.pdf.txt',
 '_index': 'parsed_resume',
 '_score': 1.0,
 '_source': {'education': {'entities': {'Entities': [{'BeginOffset': 0,
      'EndOffset': 21,
      'Score': 0.6793285012245178,
      'Text': 'Bachlors of Computing',
      'Type': 'TITLE'},
     {'BeginOffset': 23,
      'EndOffset': 30,
      'Score': 0.591768205165863,
      'Text': 'Honours',
      'Type': 'TITLE'},
     {'BeginOffset': 42,
      'EndOffset': 46,
      'Score': 0.9950048327445984,
      'Text': '2016',
      'Type': 'DATE'},
     {'BeginOffset': 51,
      'EndOffset': 69,
      'Score': 0.8884028196334839,
      'Text': "Queen's University",
      'Type': 'ORGANIZATION'},
     {'BeginOffset': 73,
      'EndOffset': 79,
      'Score': 0.9988804459571838,
      'Text': 'Canada',
      'Type': 'LOCATION'}],
    'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
      'content-length': '507',
 

In [159]:
get_user_education(user)

{'dict': defaultdict(list,
             {'DATE': ['2016'],
              'LOCATION': ['Canada'],
              'ORGANIZATION': ["Queen's University"],
              'TITLE': ['Bachlors of Computing', 'Honours']}),
 'id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_aee1411d.pdf.txt'}

In [41]:
resp_all = {}
for id in all_user.keys():
    print(id)
    user = all_user[id]
    result = get_user_education(user)
    resp_all[id] = result
    

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Updated_Resume_Ayan.pdf.txt
----- ['ORGANIZATION', 'LOCATION', 'LOCATION', 'DATE', 'QUANTITY', 'DATE', 'DATE', 'ORGANIZATION', 'LOCATION', 'QUANTITY', 'ORGANIZATION', 'LOCATION', 'LOCATION', 'DATE', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'TITLE', 'TITLE', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'LOCATION', 'LOCATION', 'DATE', 'DATE', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'LOCATION', 'LOCATION', 'DATE', 'DATE', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'LOCATION', 'DATE', 'DATE', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION', 'QUANTITY', 'ORGANIZATION', 'LOCATION', 'QUANTITY', 'ORGANIZATION']
----- ['University of Washington', 'Seattle', 'WA', '2014 May 2016', '3.76', 'Jul 2008', 'Jun 2012', 'University of P

In [27]:
resp_all['https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Updated_Resume_Ayan.pdf.txt']

{'DATE': 'Feb 2013',
 'LOCATION': 'Seattle',
 'ORGANIZATION': 'Yelp',
 'QUANTITY': '6 month',
 'TITLE': 'MySQL'}

In [8]:
for key,values in resp_all[id].items():
    print(values)

TITLE
TITLE
DATE
ORGANIZATION
LOCATION


In [61]:
def get_user_education(es, id):
    resp = {"universities": []}
    return resp

In [62]:
get_user_education(es, id)

{'universities': []}

In [94]:
get_user_education(user)

{'Text': ['Bachlors of Computing',
  'Honours',
  '2016',
  "Queen's University",
  'Canada'],
 'Type': ['TITLE', 'TITLE', 'DATE', 'ORGANIZATION', 'LOCATION']}

In [57]:
print(d_all['hits']['hits'][1]['_id'])
print(d_all['hits']['hits'][1]['_source']['education']['entities']['Entities'])

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_aee1411d.pdf.txt
[{'Score': 0.6793285012245178, 'Type': 'TITLE', 'Text': 'Bachlors of Computing', 'BeginOffset': 0, 'EndOffset': 21}, {'Score': 0.591768205165863, 'Type': 'TITLE', 'Text': 'Honours', 'BeginOffset': 23, 'EndOffset': 30}, {'Score': 0.9950048327445984, 'Type': 'DATE', 'Text': '2016', 'BeginOffset': 42, 'EndOffset': 46}, {'Score': 0.8884028196334839, 'Type': 'ORGANIZATION', 'Text': "Queen's University", 'BeginOffset': 51, 'EndOffset': 69}, {'Score': 0.9988804459571838, 'Type': 'LOCATION', 'Text': 'Canada', 'BeginOffset': 73, 'EndOffset': 79}]


In [ ]:
def extract_education(user_id):
    for i in range(len(d_all['hits']['hits'])):
        user_id = d_all['hits']['hits'][i]['_id']
        education = d_all['hits']['hits'][i]['_source']['education']['entities']['Entities']

In [18]:
es.search(index = index_name2, query=
    {
      "_source": 'education.entities.Entities.Text',
      "size": 0,
      "aggs": {
        "group_by_coll_conv": {
          "terms": {
            "field": 'education.entities.Entities.Text.keyword',
            "size" : 1000
          },
          "aggs": {
            "count" : {
              "value_count" : 
                { 
                  "field" : 'education.entities.Entities.Text.keyword'
                } 
            }
        }
    }
}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'aggregations': {'group_by_coll_conv': {'buckets': [{'count': {'value': 3543},
     'doc_count': 106,
     'key': '2015'},
    {'count': {'value': 3242}, 'doc_count': 97, 'key': '2014'},
    {'count': {'value': 2372}, 'doc_count': 93, 'key': 'India'},
    {'count': {'value': 2744}, 'doc_count': 79, 'key': '2012'},
    {'count': {'value': 1978}, 'doc_count': 78, 'key': '4.0'},
    {'count': {'value': 2271}, 'doc_count': 70, 'key': '2013'},
    {'count': {'value': 2139}, 'doc_count': 69, 'key': '2010'},
    {'count': {'value': 1861}, 'doc_count': 60, 'key': '2008'},
    {'count': {'value': 2439}, 'doc_count': 59, 'key': '2011'},
    {'count': {'value': 1370}, 'doc_count': 59, 'key': 'May 2016'},
    {'count': {'value': 1453},
     'doc_count': 59,
     'key': 'University of California'},
    {'count': {'value': 1560}, 'doc_count': 58, 'key': '2007'},
    {'count': {'value': 1895}, 'doc_count': 56, 'key': 'China'},
    {'count': {'v

In [2298]:
l_100 = count100['aggregations']['group_by_coll_conv']['buckets']

In [2299]:
keys = []
for i in l_100:
    key = i['key']
    keys.append(key)

In [2300]:
doc_counts = []
for i in l_100:
    doc_count = i['doc_count']
    doc_counts.append(doc_count)

In [2301]:
values = []
for i in l_100:
    value = i['count']['value']
    values.append(value)

In [2302]:
df = pd.DataFrame(
    {'word': keys,
     'doc_count': doc_counts,
     'values': values
    })

In [2303]:
count100['hits']['total']

622

In [2304]:
df['idf'] = np.log(count100['hits']['total']/df['doc_count'])

In [2305]:
df['tf'] = df['values']/sum(df['values'])

In [2306]:
df['tf-idf'] = df['idf']*df['tf']
df.head()

,doc_count,values,word,idf,tf,tf-idf
0,198,11712,University,1.144673,0.031296,0.035824
1,170,7781,Science,1.297142,0.020792,0.026970
2,86,4486,GPA,1.978593,0.011987,0.023718
3,74,5846,Machine Learning,2.128875,0.015621,0.033256
4,67,3077,Computer Science,2.228247,0.008222,0.018321


In [2495]:
pd.options.display.max_rows = 9999

In [2312]:
df_s = df.sort_values(by=['tf','word'], ascending=[False, True])

# all source

In [2486]:
sources = ['education', 'summary','experience','projects']

In [2440]:
results = []
for element in sources:
    result = es.search(index = index_name2, query=
    {
      "_source": element + '.keywords.KeyPhrases.Text',
      "size": 0,
      "aggs": {
        "group_by_coll_conv": {
          "terms": {
            "field": element + ".keywords.KeyPhrases.Text.keyword",
            "size" : 500
          },
          "aggs": {
            "count" : {
              "value_count" : 
                { 
                  "field" : element + ".keywords.KeyPhrases.Text.keyword"
                } 
            }
        }
    }
}})
    results.append(result)

In [2464]:
results_all = []
for i in range(len(results)):
    result_all = results[i]['aggregations']['group_by_coll_conv']['buckets']
    results_all.append(result_all)

In [ ]:
dfs = []
for j in range(len(sources)):
    source = sources[j]
    d = results_all[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': sources
        })
    dfs.append(aa)

df = pd.concat(dfs)

In [2488]:
dfs = []
for j in range(len(sources)):
    source = sources[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': source
        })
    dfs.append(aa)

df = pd.concat(dfs)
df.reset_index(drop=True)

In [2510]:
results[0]['hits']['total']

622

In [2538]:

dfs = []
for j in range(len(sources)):
    source = sources[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    for i in results_all[j]:
        idfs = np.log([results[0]['hits']['total']/x for x in doc_counts]).tolist()
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': source,
         'idf':idfs
        })
    dfs.append(aa)

df = pd.concat(dfs)
df.reset_index(drop=True)

In [2543]:
df['values']/sum(df['values'])
dfs = []
for j in range(len(sources)):
    source = sources[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    for i in results_all[j]:
        idfs = np.log([results[0]['hits']['total']/x for x in doc_counts]).tolist()
    for i in results_all[j]:
        tf = [x/sum(values) for x in values]
    for i in result_all[j]:
        tf_idf = [a*b for a,b in zip(tf,idfs)]
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': source,
         'idf':idfs,
         'tf':tf,
         'tf_idf':tf_idf
        })
    dfs.append(aa)

df = pd.concat(dfs)
df.reset_index(drop=True,inplace = True)
df= df.sort_values(by=['tf','word'], ascending=[False, True])

In [2546]:
df.head(100)

,doc_count,idf,source,tf,tf_idf,values,word
0,198,1.144673,education,0.031296,0.035824,11712,University
1000,171,1.291277,experience,0.027528,0.035547,16419,data
1500,70,2.184445,projects,0.025569,0.055854,5476,Python
1501,57,2.389889,projects,0.023314,0.055717,4993,data
1,170,1.297142,education,0.020792,0.026970,7781,Science
1001,97,1.858229,experience,0.017702,0.032894,10558,Python
7,55,2.425607,education,0.016589,0.040238,6208,data
5,58,2.372497,education,0.016329,0.038742,6111,Python
3,74,2.128875,education,0.015621,0.033256,5846,Machine Learning
1002,76,2.102207,experience,0.013423,0.028218,8006,University


In [ ]:
# save to parquet
# save to s3



In [2557]:
filename = 'terms.parquet'

In [ ]:
df.to_parquet(filename)

In [2559]:
s3.Bucket(bucketname).upload_file(filename,'sys/parsed-keywords_v1.parquet')